# Final Project

# Part 1: Predicting Base-Stacking Interactions

## Prep the data

In [0]:
!apt-get -qq install -y python-rdkit librdkit1 rdkit-data
!pip install -q joblib pandas sklearn tensorflow pillow deepchem

In [0]:
# Imports
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import pandas as pd
import io
import requests
import warnings
import random

import sklearn as sk
import rdkit as rd
import deepchem as dc
import numpy as np
import tensorflow as tf

warnings.filterwarnings("ignore")

c = pd.read_csv('final_training.csv')

W1220 03:31:22.923301 140210940082048 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Helper functions

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'Unnamed: 0', 'base_pairing', 'orientation', 'sugar_puckering', 'pseudoknot', 'stacking'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'stacking']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'stacking', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, dtype='object', columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features, dtype='object')
  info = pd.DataFrame(cs[retain].values, dtype='object', columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

In [0]:
def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'stacking', neighbors = 2, drop_names = ['id', 'stacking', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

## Set up data

In [0]:
NEIGHBORS = 1
id = '1A60'

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [0]:
cs = c
cs_all.head()

,id,stacking,resid,R-1,R,R+1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,1A60,1,1,UNK,GUA,GUA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92.7,74.9,72,82.5,65.4,153.6,109.8,157,142.1,5.65,4.5,4.12,4.23,7.469,6.124,4.07,3.99,8.413,8.03,91.9,75.7,72.4,82.3,66.1,155.207,118.377,160.7,141.4,5.88,4.6,4.5,4.31,7.83,5.549,4.2,4.15,7.871,7.49
1,1A60,1,2,GUA,GUA,GUA,92.7,74.9,72,82.5,65.4,153.6,109.8,157,142.1,5.65,4.5,4.12,4.23,7.469,6.124,4.07,3.99,8.413,8.03,91.9,75.7,72.4,82.3,66.1,155.207,118.377,160.7,141.4,5.88,4.6,4.5,4.31,7.83,5.549,4.2,4.15,7.871,7.49,92.2,76.1,72.3,84.7,66,154.079,120.289,160.7,141.2,5.77,4.5,4.09,4.19,7.776,5.651,4.1,4.08,7.904,7.24
2,1A60,1,3,GUA,GUA,ADE,91.9,75.7,72.4,82.3,66.1,155.207,118.377,160.7,141.4,5.88,4.6,4.5,4.31,7.83,5.549,4.2,4.15,7.871,7.49,92.2,76.1,72.3,84.7,66,154.079,120.289,160.7,141.2,5.77,4.5,4.09,4.19,7.776,5.651,4.1,4.08,7.904,7.24,92.2,75,72,81.2,65.8,153.7,118.8,157.085,139.1,5.96,4.57,4.53,4.54,7.52,5.932,4.28,4.11,8.413,7.68
3,1A60,1,4,GUA,ADE,GUA,92.2,76.1,72.3,84.7,66,154.079,120.289,160.7,141.2,5.77,4.5,4.09,4.19,7.776,5.651,4.1,4.08,7.904,7.24,92.2,75,72,81.2,65.8,153.7,118.8,157.085,139.1,5.96,4.57,4.53,4.54,7.52,5.932,4.28,4.11,8.413,7.68,92.2,74.8,71.8,85.7,65.4,155.07,118.2,161.165,136.1,5.63,4.43,4.4,4.18,7.793,5.988,4.18,4.07,8.522,7
4,1A60,1,5,ADE,GUA,CYT,92.2,75,72,81.2,65.8,153.7,118.8,157.085,139.1,5.96,4.57,4.53,4.54,7.52,5.932,4.28,4.11,8.413,7.68,92.2,74.8,71.8,85.7,65.4,155.07,118.2,161.165,136.1,5.63,4.43,4.4,4.18,7.793,5.988,4.18,4.07,8.522,7,93.1,76,71.8,82.2,64,159.7,96.5,141.7,141.851,5.64,4.17,4.42,4.32,7.4,5.11,4.32,4.17,7.48,8.169


## Train MLP Model

In [0]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [0]:
# build a classifier
clf = MLPClassifier(max_iter=100)

In [0]:
# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

parameter_set = {
    'hidden_layer_sizes': [(50,50,50,50)],
    'activation': ['tanh'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant'],
}

In [0]:
# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_set, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam, score=0.822596630327, total=   4.2s
[CV] alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV]  alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam, score=0.81746031746, total=   4.1s
[CV] alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.3s remaining:    0.0s


[CV]  alpha=0.0001, learning_rate=constant, hidden_layer_sizes=(50, 50, 50, 50), activation=tanh, solver=adam, score=0.849056603774, total=   4.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   12.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   12.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
          fit_params=None, iid='warn', n_iter=2, n_jobs=None,
          param_distributions={'alpha': [0.0001], 'activation': ['tanh'], 'solver': ['adam'], 'learning_rate': ['constant'], 'hidden_layer_sizes': [(50, 50, 50, 50)]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=5)

In [0]:
# Explore best parameters
# Best parameter set
print('Best parameters found:\n', random_search.best_params_)

# All results
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

('Best parameters found:\n', {'alpha': 0.0001, 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 50, 50, 50), 'activation': 'tanh', 'solver': 'adam'})
0.830 (+/-0.028) for {'alpha': 0.0001, 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 50, 50, 50), 'activation': 'tanh', 'solver': 'adam'}


In [0]:
# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.56      0.45      0.50        11
           1       0.83      0.88      0.85        33

   micro avg       0.77      0.77      0.77        44
   macro avg       0.69      0.67      0.68        44
weighted avg       0.76      0.77      0.76        44



In [0]:
# Confusion Matrix
print(confusion_matrix(y_true, y_pred))

# F1 Score --- ONE NEIGHBOR
f1_score_1 = f1_score(y_true, y_pred)
print("f1 score for 1 neighbor:", f1_score_1)

[[ 5  6]
 [ 4 29]]
('f1 score for 1 neighbor:', 0.8529411764705883)


## Random Forest Model

In [0]:
trainy=trainy.astype('int')

from sklearn.ensemble import RandomForestClassifier
model_numba2 = RandomForestClassifier(max_depth=10, random_state=0, verbose=2)
model_numba2.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=2, warm_start=False)

In [0]:
y_true, y_pred = np.int_(testy) , model_numba2.predict(testX_scaled)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

f1_score_1 = f1_score(y_true, y_pred)
print("f1 score for 1 neighbor:", f1_score_1)

Results on the test set:
              precision    recall  f1-score   support

           0       1.00      0.18      0.31        11
           1       0.79      1.00      0.88        33

   micro avg       0.80      0.80      0.80        44
   macro avg       0.89      0.59      0.59        44
weighted avg       0.84      0.80      0.74        44

('f1 score for 1 neighbor:', 0.88)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


# Part 2: SASA

1. Deepchem Multitask Regressor
2. Models for individual SASA tasks

  i.   Random Forests

  ii.   MLPs

## Imports

In [0]:
!apt-get -qq install -y python-rdkit librdkit1 rdkit-data
!pip install -q joblib pandas sklearn tensorflow pillow deepchem

In [0]:
# Imports
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import f1_score
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import io
import requests
import warnings
import random

import sklearn as sk
import rdkit as rd
import tensorflow as tf
import deepchem as dc

warnings.filterwarnings("ignore")  

## Deepchem Multitask Regressor Model

### Loads and Preps the data

In [0]:
# loads csv with SASA data
d = pd.read_csv('final_training_sasa_outputs.csv')
d = d.rename(columns={"id": "sasa_id", "resname": "sasa_resname", "resid": "sasa_resid"})

# loads csv with chemical shifts
c = pd.read_csv('final_training.csv')

# concatenates the two into single dataframe
e = pd.concat([c, d], axis=1, sort=False)
e['resid'] = e['sasa_resid'].astype(int)
c = e

### Helper Functions

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'Unnamed: 0', 'base_pairing', 'orientation', 'sugar_puckering', 'pseudoknot', 'stacking', 'sasa_id', 'sasa_resname', 'sasa_resid', 'sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, dtype='object', columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features, dtype='object')
  info = pd.DataFrame(cs[retain].values, dtype='object', columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

### Deepchem Dataset Generator functions

(based on incomplete source code)

In [0]:
def pd_featurize(input_files, dataset_obj, data_dir=None, shard_size=8192):
  """Featurize provided files and write to specified location.
  For large datasets, automatically shards into smaller chunks
  for convenience.
  Parameters
  ----------
  input_files: list
    List of input filenames.
  data_dir: str
    (Optional) Directory to store featurized dataset.
  shard_size: int
    (Optional) Number of examples stored in each shard.
  """
  import os
  import gzip
  import pandas as pd
  import numpy as np
  import csv
  import numbers
  import tempfile
  import time
  import sys
  import deepchem
  from deepchem.utils.save import log
  from deepchem.utils.save import load_csv_files
  from deepchem.utils.save import load_sdf_files
  from deepchem.utils.genomics import encode_fasta_sequence
  from deepchem.feat import UserDefinedFeaturizer
  from deepchem.data import DiskDataset, NumpyDataset, ImageDataset

  log("Loading raw samples now.", dataset_obj.verbose)
  tasks = ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'] #list of tasks

  def featurize_data():
    '''
    New function that takes in the global input pandas dataframe and returns X, y, w, ids
    numpy arrays to be fed into the deepchem dataset creator function.
    '''
    # copies the input pandas dataset
    copyDataset = input_files
    # creates a list of columns to be dropped (so that only chemical shifts fed to X)
    drop_names = ['id', 'sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar', 'resid']
    drop_names = drop_names = drop_names + list(write_out_resname(0))
    # creates a list of columns for the model to predict values of
    target_names = ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar']
    # separates the X and y features for the dataset
    X = copyDataset.drop(drop_names, axis=1)
    y = copyDataset[target_names]
    index = np.arange(19)
    # converts the pandas dataframes to numpy arrays
    X, idk = deepchem.data.data_loader.convert_df_to_numpy(X,index)
    ids = np.arange(len(X))
    y, w = deepchem.data.data_loader.convert_df_to_numpy(y,tasks)
    # changes the NaN values in the numpy arrays into zeroes
    X = np.nan_to_num(X)
    y = np.nan_to_num(y)
    
    # returns the X, y, w, ids features for the dataset
    yield X, y, w, ids


  def shard_generator():
    '''
    Incomplete shard_generator function from featurize function in DataLoader class
    from deepchem.data.data_loader source code.
    Not used here.
    '''
    shard = input_files
    time1 = time.time()
    X, y = dataset_obj.featurize_data(shard)
    ids = shard[dataset_obj.id_field].values
    ids = ids[valid_inds]
    if len(dataset_obj.tasks) > 0:
      # Featurize task results iff they exist.
      y, w = convert_df_to_numpy(shard, dataset_obj.tasks, dataset_obj.id_field)
      # Filter out examples where featurization failed.
      y, w = (y[valid_inds], w[valid_inds])
      assert len(X) == len(ids) == len(y) == len(w)
    else:
      # For prospective data where results are unknown, it makes
      # no sense to have y values or weights.
      y, w = (None, None)
      assert len(X) == len(ids)
    time2 = time.time()
    log(
        "TIMING: featurizing shard %d took %0.3f s" %
        (shard_num, time2 - time1), dataset_obj.verbose)
    yield X, y, w, ids

  # creates and returns a deepchem datset using the X, y, w and ids features returned
  # from the above featurize_data function
  return DiskDataset.create_dataset(
      featurize_data(), data_dir, tasks, verbose=dataset_obj.verbose)

In [0]:
def pd_data_loader(data):
  '''
  Loads in data from a pandas dataframe into a deepchem dataset and separates
  it into train, valid, and test datasets.
  '''

  import os
  import logging
  import deepchem
  from deepchem.utils.save import log

  # load all the RNA features as a pandas dataframe
  cs_all = get_cs_features_rna_all(data, neighbors = 0)
  
  # loads the pandas dataframe into a deepchem dataset
  tasks = ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'] #list of tasks
  loader = deepchem.data.data_loader.DataLoader(tasks)
  dataset = pd_featurize(cs_all, loader)

  print("[INFO]: created training and testing data structures")

  # split the data into train, valid, and test sets
  splitters = {
      'index': deepchem.splits.IndexSplitter(),
      'random': deepchem.splits.RandomSplitter(),
      'scaffold': deepchem.splits.ScaffoldSplitter(),
      'stratified': deepchem.splits.SingletaskStratifiedSplitter()
  }
  split = 'random'
  splitter = splitters[split]
  log("About to split dataset with {} splitter.".format(split))
  train, valid, test = splitter.train_valid_test_split(dataset)

  # transform the data
  transformers = []

  # return tasks, (train, valid, test sets), and transformer
  return tasks, (train, valid, test), transformers

### Multitask Regressor

Loads the data and trains a Multitask Regressor

In [29]:
# loads and splits the data
tasks, datasets, transformers = pd_data_loader(c)
train_dataset, valid_dataset, test_dataset = datasets

# creates and trains a multitask regressor for predicting the 5 SASA tasks based on the 19 chemical shifts
model = dc.models.MultitaskRegressor(n_tasks=5, n_features=19, layer_sizes=[1000, 1000, 500, 400, 300, 200, 100, 10], dropouts=.5)
model.fit(train_dataset, nb_epoch=10)

Loading raw samples now.


W1220 03:35:04.920492 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/tensor_graph.py:715: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1220 03:35:04.928037 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/layers.py:2464: The name tf.FIFOQueue is deprecated. Please use tf.queue.FIFOQueue instead.

W1220 03:35:04.936079 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/layers.py:1216: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1220 03:35:04.952599 140210940082048 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be remov

TIMING: dataset construction took 1.184 s
Loading dataset from disk.
[INFO]: created training and testing data structures
About to split dataset with random splitter.
TIMING: dataset construction took 0.031 s
Loading dataset from disk.
TIMING: dataset construction took 0.013 s
Loading dataset from disk.
TIMING: dataset construction took 0.012 s
Loading dataset from disk.


W1220 03:35:05.147907 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/tensor_graph.py:728: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1220 03:35:05.168153 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/optimizers.py:76: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W1220 03:35:05.740736 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/tensor_graph.py:1013: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W1220 03:35:05.741990 140210940082048 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/deepchem/models/tensorgraph/tensor_graph.py:1013: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1220 03:35:05.748630 1402109400820

531232.116625

Metrics of model

In [30]:
# computes the Pearson R2 score for each of the SASA tasks for the train dataset and the test dataset
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Model Tasks: ", ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'])
print(model.evaluate(train_dataset, [metric], transformers))
print("")
print("Model Tasks: ", ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'])
print(model.evaluate(test_dataset, [metric], transformers))

('Model Tasks: ', ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'])
computed_metrics: [0.01609697823504552, 0.007637485210277998, 0.011541177070959432, 0.009949218532251584, 0.012528712596440738]
{'pearson_r2_score': [0.01609697823504552, 0.007637485210277998, 0.011541177070959432, 0.009949218532251584, 0.012528712596440738]}

('Model Tasks: ', ['sasa-All-atoms', 'sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'])
computed_metrics: [0.01548135698143717, 0.010973720438411612, 0.006431489508091624, 0.006779608397916491, 0.014029525890830662]
{'pearson_r2_score': [0.01548135698143717, 0.010973720438411612, 0.006431489508091624, 0.006779608397916491, 0.014029525890830662]}


## Individual Tasks Approach

In [0]:
# loads csv file with all data
c = pd.read_csv('final_training_sasa.csv')

### SASA-ALL-ATOMS

#### Helper Functions

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'sasa-All-atoms',	'sasa-Total-Side'	, 'sasa-Main-Chain'	,'sasa-Non-polar',	'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-All-atoms']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-All-atoms', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features)
  info = pd.DataFrame(cs[retain].values, columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'sasa-All-atoms', neighbors = 2, drop_names = ['id', 'sasa-All-atoms', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [33]:
NEIGHBORS = 1
id = '1A60' #NEED TO FIGURE OUT WHY THE ID MUST MATCH THAT ABOVE/WHY CAN'T SELECT ANY ID

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [34]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [0]:
#trainX = np.nan_to_num(trainX)
trainX_scaled = np.nan_to_num(trainX_scaled)
#trainX_scaled = trainX_scaled[~np.isnan(trainX_scaled)]

In [0]:
# build a classifier
clf = MLPRegressor(max_iter=100)

In [0]:
# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

In [38]:
# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] solver=sgd, activation=relu, hidden_layer_sizes=(25,), alpha=0.008189219988522624, learning_rate=adaptive, learning_rate_init=0.000657164273581347 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  solver=sgd, activation=relu, hidden_layer_sizes=(25,), alpha=0.008189219988522624, learning_rate=adaptive, learning_rate_init=0.000657164273581347, score=0.253945466728, total=   0.6s
[CV] solver=sgd, activation=relu, hidden_layer_sizes=(25,), alpha=0.008189219988522624, learning_rate=adaptive, learning_rate_init=0.000657164273581347 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  solver=sgd, activation=relu, hidden_layer_sizes=(25,), alpha=0.008189219988522624, learning_rate=adaptive, learning_rate_init=0.000657164273581347, score=0.321195416391, total=   0.6s
[CV] solver=sgd, activation=relu, hidden_layer_sizes=(25,), alpha=0.008189219988522624, learning_rate=adaptive, learning_rate_init=0.000657164273581347 


...

[CV]  solver=adam, activation=tanh, hidden_layer_sizes=(49, 20, 51, 12), alpha=0.0027539481910834095, learning_rate=adaptive, learning_rate_init=0.0007566979897006438, score=-19.0986675605, total=   3.1s
[CV] solver=adam, activation=tanh, hidden_layer_sizes=(49, 20, 51, 12), alpha=0.0027539481910834095, learning_rate=adaptive, learning_rate_init=0.0007566979897006438 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    5.1s remaining:    0.0s


[CV]  solver=adam, activation=tanh, hidden_layer_sizes=(49, 20, 51, 12), alpha=0.0027539481910834095, learning_rate=adaptive, learning_rate_init=0.0007566979897006438, score=-17.5766529428, total=   3.0s
[CV] solver=adam, activation=tanh, hidden_layer_sizes=(49, 20, 51, 12), alpha=0.0027539481910834095, learning_rate=adaptive, learning_rate_init=0.0007566979897006438 
[CV]  solver=adam, activation=tanh, hidden_layer_sizes=(49, 20, 51, 12), alpha=0.0027539481910834095, learning_rate=adaptive, learning_rate_init=0.0007566979897006438, score=-25.4807340576, total=   3.1s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   11.2s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
          fit_params=None, iid='warn', n_iter=2, n_jobs=None,
          param_distributions={'solver': ['sgd', 'adam', 'lbfgs'], 'activation': ['tanh', 'relu'], 'hidden_layer_sizes': [(49, 20, 51, 12), (70, 58, 93), (43, 93), (25,)], 'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f84f5e65350>, 'learning_rate': ['constant', 'adaptive'], 'learning_rate_init': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f84efccfe50>},
          pre_

In [39]:
# Explore best parameters
# Best parameter set
print('Best parameters found:\n', random_search.best_params_)

# All results
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

('Best parameters found:\n', {'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (25,), 'alpha': 0.008189219988522624, 'learning_rate': 'adaptive', 'learning_rate_init': 0.000657164273581347})
0.306 (+/-0.075) for {'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (25,), 'alpha': 0.008189219988522624, 'learning_rate': 'adaptive', 'learning_rate_init': 0.000657164273581347}
-20.719 (+/-6.848) for {'solver': 'adam', 'activation': 'tanh', 'hidden_layer_sizes': (49, 20, 51, 12), 'alpha': 0.0027539481910834095, 'learning_rate': 'adaptive', 'learning_rate_init': 0.0007566979897006438}


In [40]:
# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(r2_score(y_true, y_pred))

Results on the test set:
0.062474026875103394


#### Random Forest Model

In [41]:
from sklearn.ensemble import RandomForestRegressor

trainy=trainy.astype('int')

model = RandomForestRegressor(max_depth=10, random_state=0, verbose=2)
model.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=2, warm_start=False)

In [42]:
print("R2 score for Random Forest:")
model.score(testX_scaled, testy)

R2 score for Random Forest:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.12815280590861233

### SASA-TOTAL-SIDE


In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'sasa-All-atoms',	'sasa-Total-Side'	, 'sasa-Main-Chain'	,'sasa-Non-polar',	'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-Total-Side']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-Total-Side', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features)
  info = pd.DataFrame(cs[retain].values, columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'sasa-Total-Side', neighbors = 2, drop_names = ['id', 'sasa-Total-Side', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [44]:
NEIGHBORS = 1
id = '1A60' #NEED TO FIGURE OUT WHY THE ID MUST MATCH THAT ABOVE/WHY CAN'T SELECT ANY ID

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [45]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [46]:
trainX_scaled = np.nan_to_num(trainX_scaled)

# build a classifier
clf = MLPRegressor(max_iter=1000)

# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(r2_score(y_true, y_pred))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(45, 87), alpha=0.00231413506448271, learning_rate=adaptive, learning_rate_init=0.0008546381834146066 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(45, 87), alpha=0.00231413506448271, learning_rate=adaptive, learning_rate_init=0.0008546381834146066, score=0.110366644029, total=   8.9s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(45, 87), alpha=0.00231413506448271, learning_rate=adaptive, learning_rate_init=0.0008546381834146066 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.9s remaining:    0.0s


[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(45, 87), alpha=0.00231413506448271, learning_rate=adaptive, learning_rate_init=0.0008546381834146066, score=0.177052653046, total=   5.7s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(45, 87), alpha=0.00231413506448271, learning_rate=adaptive, learning_rate_init=0.0008546381834146066 


...

[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(72, 6, 47, 78), alpha=0.007543185359823325, learning_rate=constant, learning_rate_init=6.176346444157013e-05, score=-0.00721595958877, total=  10.6s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(72, 6, 47, 78), alpha=0.007543185359823325, learning_rate=constant, learning_rate_init=6.176346444157013e-05 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   51.4s remaining:    0.0s


[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(72, 6, 47, 78), alpha=0.007543185359823325, learning_rate=constant, learning_rate_init=6.176346444157013e-05, score=0.147053166678, total=   8.0s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(72, 6, 47, 78), alpha=0.007543185359823325, learning_rate=constant, learning_rate_init=6.176346444157013e-05 
[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(72, 6, 47, 78), alpha=0.007543185359823325, learning_rate=constant, learning_rate_init=6.176346444157013e-05, score=-0.0534148766011, total=   7.8s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.1min finished


Results on the test set:
-0.5445081042765052


### Random Forest Regressor

In [47]:
from sklearn.ensemble import RandomForestRegressor

trainy=trainy.astype('int')

model = RandomForestRegressor(max_depth=10, random_state=0, verbose=2)
model.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=2, warm_start=False)

In [48]:
print("R2 score for Random Forest:")
model.score(testX_scaled, testy)

R2 score for Random Forest:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


-0.061576470404669965

### SASA-MAIN-CHAIN

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'sasa-All-atoms',	'sasa-Total-Side'	, 'sasa-Main-Chain'	,'sasa-Non-polar',	'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-Main-Chain']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-Main-Chain', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features)
  info = pd.DataFrame(cs[retain].values, columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'sasa-Main-Chain', neighbors = 2, drop_names = ['id', 'sasa-Main-Chain', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [50]:
NEIGHBORS = 1
id = '1A60' #NEED TO FIGURE OUT WHY THE ID MUST MATCH THAT ABOVE/WHY CAN'T SELECT ANY ID

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [51]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
trainX_scaled = np.nan_to_num(trainX_scaled)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [52]:

# build a classifier
clf = MLPRegressor(max_iter=1000)

# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(r2_score(y_true, y_pred))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] solver=adam, activation=relu, hidden_layer_sizes=(98, 84, 88, 34), alpha=0.01243625321386744, learning_rate=constant, learning_rate_init=0.0006257809218790376 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  solver=adam, activation=relu, hidden_layer_sizes=(98, 84, 88, 34), alpha=0.01243625321386744, learning_rate=constant, learning_rate_init=0.0006257809218790376, score=-0.0626949039685, total=  16.0s
[CV] solver=adam, activation=relu, hidden_layer_sizes=(98, 84, 88, 34), alpha=0.01243625321386744, learning_rate=constant, learning_rate_init=0.0006257809218790376 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.1s remaining:    0.0s


[CV]  solver=adam, activation=relu, hidden_layer_sizes=(98, 84, 88, 34), alpha=0.01243625321386744, learning_rate=constant, learning_rate_init=0.0006257809218790376, score=0.00403662768157, total=  13.0s
[CV] solver=adam, activation=relu, hidden_layer_sizes=(98, 84, 88, 34), alpha=0.01243625321386744, learning_rate=constant, learning_rate_init=0.0006257809218790376 


...

[CV]  solver=lbfgs, activation=relu, hidden_layer_sizes=(80,), alpha=0.0046373589345600355, learning_rate=adaptive, learning_rate_init=0.0015804855194320373, score=-1.11767241102, total=   6.7s
[CV] solver=lbfgs, activation=relu, hidden_layer_sizes=(80,), alpha=0.0046373589345600355, learning_rate=adaptive, learning_rate_init=0.0015804855194320373 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   52.3s remaining:    0.0s


[CV]  solver=lbfgs, activation=relu, hidden_layer_sizes=(80,), alpha=0.0046373589345600355, learning_rate=adaptive, learning_rate_init=0.0015804855194320373, score=-2.57569537384, total=   6.7s
[CV] solver=lbfgs, activation=relu, hidden_layer_sizes=(80,), alpha=0.0046373589345600355, learning_rate=adaptive, learning_rate_init=0.0015804855194320373 
[CV]  solver=lbfgs, activation=relu, hidden_layer_sizes=(80,), alpha=0.0046373589345600355, learning_rate=adaptive, learning_rate_init=0.0015804855194320373, score=-1.59535258779, total=   6.7s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.1min finished


Results on the test set:
-0.6255357226298663


Random Forest Regressor

In [53]:
from sklearn.ensemble import RandomForestRegressor

trainy=trainy.astype('int')

model = RandomForestRegressor(max_depth=10, random_state=0, verbose=2)
model.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=2, warm_start=False)

In [54]:
print("R2 score for Random Forest:")
model.score(testX_scaled, testy)

R2 score for Random Forest:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.3219866870482875

### SASA-NON-POLAR

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'sasa-All-atoms',	'sasa-Total-Side'	, 'sasa-Main-Chain'	,'sasa-Non-polar',	'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-Non-polar']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-Non-polar', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features)
  info = pd.DataFrame(cs[retain].values, columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'sasa-Non-polar', neighbors = 2, drop_names = ['id', 'sasa-Non-polar', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [56]:
NEIGHBORS = 1
id = '1A60'

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [57]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
trainX_scaled = np.nan_to_num(trainX_scaled)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [58]:
# build a classifier
clf = MLPRegressor(max_iter=1000)

# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(r2_score(y_true, y_pred))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(76, 35), alpha=0.020327676759129328, learning_rate=adaptive, learning_rate_init=0.000991779069950693 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(76, 35), alpha=0.020327676759129328, learning_rate=adaptive, learning_rate_init=0.000991779069950693, score=0.278333530949, total=   9.2s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(76, 35), alpha=0.020327676759129328, learning_rate=adaptive, learning_rate_init=0.000991779069950693 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s remaining:    0.0s


[CV]  solver=sgd, activation=tanh, hidden_layer_sizes=(76, 35), alpha=0.020327676759129328, learning_rate=adaptive, learning_rate_init=0.000991779069950693, score=0.101436772509, total=  13.4s
[CV] solver=sgd, activation=tanh, hidden_layer_sizes=(76, 35), alpha=0.020327676759129328, learning_rate=adaptive, learning_rate_init=0.000991779069950693 


...

[CV]  solver=sgd, activation=relu, hidden_layer_sizes=(76, 35), alpha=0.014649460553345369, learning_rate=adaptive, learning_rate_init=0.00037572641053988263, score=0.327473546225, total=  10.1s
[CV] solver=sgd, activation=relu, hidden_layer_sizes=(76, 35), alpha=0.014649460553345369, learning_rate=adaptive, learning_rate_init=0.00037572641053988263 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   48.7s remaining:    0.0s


[CV]  solver=sgd, activation=relu, hidden_layer_sizes=(76, 35), alpha=0.014649460553345369, learning_rate=adaptive, learning_rate_init=0.00037572641053988263, score=0.00741031563203, total=  12.1s
[CV] solver=sgd, activation=relu, hidden_layer_sizes=(76, 35), alpha=0.014649460553345369, learning_rate=adaptive, learning_rate_init=0.00037572641053988263 
[CV]  solver=sgd, activation=relu, hidden_layer_sizes=(76, 35), alpha=0.014649460553345369, learning_rate=adaptive, learning_rate_init=0.00037572641053988263, score=0.180963205594, total=   8.7s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.2min finished


Results on the test set:
-0.40919272580802435


### Random Forest Regressor

In [59]:
from sklearn.ensemble import RandomForestRegressor

trainy=trainy.astype('int')

model = RandomForestRegressor(max_depth=10, random_state=0, verbose=2)
model.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=2, warm_start=False)

In [60]:
print("R2 score for Random Forest:")
model.score(testX_scaled, testy)

R2 score for Random Forest:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.14450419702885586

### SASA-ALL-POLAR

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'sasa-All-atoms',	'sasa-Total-Side'	, 'sasa-Main-Chain'	,'sasa-Non-polar',	'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-All-polar']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-All-polar', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features)
  info = pd.DataFrame(cs[retain].values, columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
  '''
  
  cs_new = pd.DataFrame()
  for i in range(0,len(c.id.unique())):
    csData = get_cs_features_rna(get_cs_all(cs, c.id.unique()[i]), neighbors)
    cs_new = pd.concat([cs_new, csData], axis=0) 
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = '2KOC', target_name = 'sasa-All-polar', neighbors = 2, drop_names = ['id', 'sasa-All-polar', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [62]:
NEIGHBORS = 1
id = '1A60'

cs_all = get_cs_features_rna_all(c, neighbors = NEIGHBORS)

trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

[INFO]: created training and testing data structures


In [63]:
# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
trainX_scaled = np.nan_to_num(trainX_scaled)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: scaled the features


In [64]:
# build a classifier
clf = MLPRegressor(max_iter=1000)

# Specify parameters to choose from
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

# Setup and run randomized search
n_iter_search = 2
random_search = RandomizedSearchCV(clf, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=3, verbose = 5)

random_search.fit(trainX_scaled, np.int_(trainy))

# Make predictions on test set
y_true, y_pred = np.int_(testy) , random_search.predict(testX_scaled)
print('Results on the test set:')
print(r2_score(y_true, y_pred))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] solver=adam, activation=tanh, hidden_layer_sizes=(49,), alpha=0.019108088305276313, learning_rate=adaptive, learning_rate_init=3.877211058234038e-05 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  solver=adam, activation=tanh, hidden_layer_sizes=(49,), alpha=0.019108088305276313, learning_rate=adaptive, learning_rate_init=3.877211058234038e-05, score=-16.5008273697, total=  12.6s
[CV] solver=adam, activation=tanh, hidden_layer_sizes=(49,), alpha=0.019108088305276313, learning_rate=adaptive, learning_rate_init=3.877211058234038e-05 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.6s remaining:    0.0s


[CV]  solver=adam, activation=tanh, hidden_layer_sizes=(49,), alpha=0.019108088305276313, learning_rate=adaptive, learning_rate_init=3.877211058234038e-05, score=-13.2584305328, total=  12.6s
[CV] solver=adam, activation=tanh, hidden_layer_sizes=(49,), alpha=0.019108088305276313, learning_rate=adaptive, learning_rate_init=3.877211058234038e-05 


...

[CV]  solver=adam, activation=relu, hidden_layer_sizes=(58, 35, 77), alpha=0.006631566350992895, learning_rate=constant, learning_rate_init=0.0006865467290116567, score=-0.0119797274951, total=  10.1s
[CV] solver=adam, activation=relu, hidden_layer_sizes=(58, 35, 77), alpha=0.006631566350992895, learning_rate=constant, learning_rate_init=0.0006865467290116567 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   47.9s remaining:    0.0s


[CV]  solver=adam, activation=relu, hidden_layer_sizes=(58, 35, 77), alpha=0.006631566350992895, learning_rate=constant, learning_rate_init=0.0006865467290116567, score=0.112914450955, total=  12.5s
[CV] solver=adam, activation=relu, hidden_layer_sizes=(58, 35, 77), alpha=0.006631566350992895, learning_rate=constant, learning_rate_init=0.0006865467290116567 
[CV]  solver=adam, activation=relu, hidden_layer_sizes=(58, 35, 77), alpha=0.006631566350992895, learning_rate=constant, learning_rate_init=0.0006865467290116567, score=-0.0999091658404, total=  10.8s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.2min finished


Results on the test set:
-0.2671503495553025


### Random Forest Regressor

In [65]:
from sklearn.ensemble import RandomForestRegressor

trainy=trainy.astype('int')

model = RandomForestRegressor(max_depth=10, random_state=0, verbose=2)
model.fit(trainX_scaled, trainy)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=2, warm_start=False)

In [66]:
print("R2 score for Random Forest:")
model.score(testX_scaled, testy)

R2 score for Random Forest:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


-0.04256304319648696